# **AI LAB 2023**
### A. Calderoni, A. Fabbricatore, S. Mancini, A. Pranzo, B. Zanini

## Experimental Background

**Definitions**
- Hypoxia: "depleted" oxygen levels in tissue culture flasks, i.e. about 2% oxygen
- Normoxia: “normal” oxygen levels in tissue culture flasks, i.e. about 20–21% oxygen

**Why is it important to study whether cells are hypoxia or normoxia?**


**Data Experimental Design**

**Data Structure**

**Aim of our Project**


**Why cancer and genome?**

Studying the correlation between cancer and human genome is a difficult thing because the gene expression of cancer cells has an high variability among people and among the cells of the single person. Moreover, there can be many mutations developing over the time. If we want to understand and cure cancer cells is crucial to study their interactions with their neighborhoods (to see how cancer cells are escaping the T-cells, for example). Because of that we can't limit ourselfs to the study of the DNA, but we need to study the RNA since it is the "instruction list" that guides the cell behaviour/methabolism.

**Why is it important to study hypoxia?** 

Cancer cells are excessively grown up cells that refuse to die committing a "controlled suicide" (*apoptosis*) and instead continue to grow in your body. To do that they'll need more oxigen than the normal cells and in the regions with lage metastasis the blood vassels will not be able to keep up whith the cancer cells request of oxigen. The result is that cancer cells, about to die "against their will", will end up expressing abnormal genes (to encode a metabolic reaction to survive these extreme coditions) that eventually will lead them to become much more aggressive and difficult to treat.
Our question now is: can we identify apoxic cells? If we could understand how they react we could better treat cancer!

**How is the experiment designed?**

In our experiemnt we grow some cells in two different environments, one whith normal levels of oxigen (21%) and others with reduced levels of oxigen (1%) of oxigen. We then sequenced every single cell (Single-Cell RNA sequencing using the Smart-Seq and the Drop-Seq methods). Our target is starting from data to build a good predictor that given the gene expression of a celll will tell if this last was living in an hypoxic or normal environment. Both these two sequecing techniques were applied to two different cell lines, called HCC1086 and MCF7. The first one is a type of liver cancer and the second one is breast cancer.

**How are the data structured?**

The data provided to us were given in the format of .csv table. Every column represents a single cell sequenced (so an observetion), identified with a precise name containig informations about which of the two treatment conditions it grew up in (hypoxia or normoxia) and in case of Smarseq also its position on the plate of colture. Every row represents a single gene (so a feature), identified with its official gene symbol. Each entry of the table represents the gene expression count in case of SmartSeq sequenced cells and the Unique Molecular Identifier count (UMI) in case of DropSeq sequenced cells.

**How will we proceed with these data?**
We will analyse all the four tables given to us using a pretty standard Differential Gene Expression study pipeline that will include the following:

- Exploratory data analysis and Data cleaning
- Quality Control and Nomralization
- Feature selection and Higly Variable Genes study
- Dimensionality Reduction and Clustering 
- Supervised learning: Tree based methods and Deep Neural Networks
- BONUS: Heuristics on Enrichment Analysis and Ontology Analysis

## Exploratory Data Analysis and Cleaning

## Dimensionality Reduction

## Model